In [69]:
import json

def are_spans_overlapping(spans, text):
    # Sort the spans based on the start index
    sorted_spans = sorted(spans, key=lambda x: x[0])
    for i in range(1, len(sorted_spans)):
        # Check if the current span overlaps with the previous span
        if sorted_spans[i][0] < sorted_spans[i - 1][1]:
            print('overlapping spans:',sorted_spans[i-1],sorted_spans[i])
            print('first span text:',text[sorted_spans[i-1][0]:sorted_spans[i-1][1]])
            print('second span text:',text[sorted_spans[i][0]:sorted_spans[i][1]])
            overlapping_text = text[sorted_spans[i-1][0]:sorted_spans[i][1]]
            print('overlapping text:',overlapping_text)
            return True

    return False

def remove_overlapping_spans(spans):
    sorted_spans = sorted(spans, key=lambda x: x[0])
    pruned_spans = [sorted_spans[0]]

    for span in sorted_spans[1:]:
        if span[0] >= pruned_spans[-1][1]:
            pruned_spans.append(span)

    return pruned_spans

def fix_overlaps(spans, text):
    sorted_spans = sorted(spans, key=lambda x: x[0])
    for i in range(1, len(sorted_spans)):
        
        # Check if the current span overlaps with the previous span
        if sorted_spans[i][0] < sorted_spans[i - 1][1]:
            overlapping_text = text[sorted_spans[i-1][0]:sorted_spans[i][1]]
            split_text = overlapping_text.split()
            first_new_span_text = split_text[0]
            second_new_span_text = ' '.join(split_text[1:])
            first_new_span = [sorted_spans[i-1][0],sorted_spans[i-1][0]+len(first_new_span_text), sorted_spans[i-1][2]]
            second_new_span = [sorted_spans[i-1][0]+len(first_new_span_text)+1,sorted_spans[i][1], sorted_spans[i][2]]
            # reassign the spans
            spans[i-1] = first_new_span
            spans[i] = second_new_span
            # return True

    
    # Step 2: Remove duplicates and keep enclosed spans
    unique_spans = []
    for span in sorted_spans:
        if span not in unique_spans:
            # Check if the span is enclosed in any existing span
            enclosed = False
            for existing_span in unique_spans:
                if span[0] >= existing_span[0] and span[1] <= existing_span[1]:
                    enclosed = True
                    break
            if not enclosed:
                # Remove any existing span that is enclosed by the new span
                unique_spans = [existing_span for existing_span in unique_spans if not (existing_span[0] >= span[0] and existing_span[1] <= span[1])]
                unique_spans.append(span)

    return unique_spans

path = '/home/pgajo/working/food/data/TASTEset/data/TASTEset_semicolon_formatted_en-it_unaligned_aligned_model=mdeberta-v3-base-xl-wa_recipe_aligner_5epochs_error_rate=0.0119.json'
new_path = path.replace('.json', '_pruned.json')

with open(path) as f:
    data = json.load(f)

error_count = 0
for i, recipe in enumerate(data['annotations']):
    recipe['entities_it'] = fix_overlaps(recipe['entities_it'], recipe['text_it'])
    pruned_spans = remove_overlapping_spans(recipe['entities_it'])

    if pruned_spans != recipe['entities_it']:
        print('Overlap removed at', i)
        recipe['entities_it'] = pruned_spans
        error_count += 1

print('Number of recipes with removed overlaps:', error_count)

# Save the adjusted data to a new JSON file
with open(new_path, 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Data saved to {new_path}")

Overlap removed at 35
Overlap removed at 144
Overlap removed at 184
Overlap removed at 256
Overlap removed at 389
Overlap removed at 418
Overlap removed at 439
Overlap removed at 499
Number of recipes with removed overlaps: 8
Data saved to /home/pgajo/working/food/data/TASTEset/data/TASTEset_semicolon_formatted_en-it_unaligned_aligned_model=mdeberta-v3-base-xl-wa_recipe_aligner_5epochs_error_rate=0.0119_pruned.json


In [64]:
import json

def are_spans_overlapping(spans, text):
    # Sort the spans based on the start index
    sorted_spans = sorted(spans, key=lambda x: x[0])
    # print('text:',text)
    # print('sorted_spans:',sorted_spans)
    # Iterate through the sorted spans
    for i in range(1, len(sorted_spans)):
        # Check if the current span overlaps with the previous span
        if sorted_spans[i][0] < sorted_spans[i - 1][1]:
            print('overlapping spans:',sorted_spans[i-1],sorted_spans[i])
            print('first span text:',text[sorted_spans[i-1][0]:sorted_spans[i-1][1]])
            print('second span text:',text[sorted_spans[i][0]:sorted_spans[i][1]])
            overlapping_text = text[sorted_spans[i-1][0]:sorted_spans[i][1]]
            print('overlapping text:',overlapping_text)
            return True

    return False

def fix_overlaps(spans, text):
    # Sort the spans based on the start index
    # print('---fix_overlaps---')
    sorted_spans = sorted(spans, key=lambda x: x[0])
    # print('pre-fix spans:',spans)
    # Iterate through the sorted spans
    for i in range(1, len(sorted_spans)):
        
        # Check if the current span overlaps with the previous span
        if sorted_spans[i][0] < sorted_spans[i - 1][1]:
            # print('*** overlapping at index:',i, '***')
            # print('overlapping old spans:',sorted_spans[i-1],sorted_spans[i])
            overlapping_text = text[sorted_spans[i-1][0]:sorted_spans[i][1]]
            # print('overlapping text:',overlapping_text)
            # print('old first span text:',text[sorted_spans[i-1][0]:sorted_spans[i-1][1]])
            # print('old second span text:',text[sorted_spans[i][0]:sorted_spans[i][1]])
            split_text = overlapping_text.split()
            first_new_span_text = split_text[0]
            second_new_span_text = ' '.join(split_text[1:])
            # print('first new span text:',first_new_span_text)
            # print('second new span text:',second_new_span_text)
            first_new_span = [sorted_spans[i-1][0],sorted_spans[i-1][0]+len(first_new_span_text), sorted_spans[i-1][2]]
            second_new_span = [sorted_spans[i-1][0]+len(first_new_span_text)+1,sorted_spans[i][1], sorted_spans[i][2]]
            
            # reassign the spans
            spans[i-1] = first_new_span
            spans[i] = second_new_span
            # print('new spans:',spans[i-1],spans[i])
            # print('check first new span:',text[spans[i-1][0]:spans[i-1][1]])
            # print('check second new span:',text[spans[i][0]:spans[i][1]])
            
            # return True

    return False

path = '/home/pgajo/working/food/data/TASTEset/data/TASTEset_semicolon_formatted_en-it_unaligned_aligned_model=mdeberta-v3-base-xl-wa_recipe_aligner_5epochs_error_rate=0.0119.json'
with open(path) as f:
    data = json.load(f)
# print(len(data['annotations']))
error_count = 0
for i, recipe in enumerate(data['annotations']):
    # print('***', i)
    # # print("recipe['text_en']:",recipe['text_en'])
    # fix_overlaps(recipe['entities_en'], recipe['text_en'])
    # if are_spans_overlapping(recipe['entities_en']):
    #     print('error at', i)
    #     # print(recipe['entities_en'])
    #     error_count += 1
    # # for entity in recipe['entities_en']:
    # #     print(recipe['text_en'][entity[0]:entity[1]],end=' ')
    # # print()
    # # print("recipe['text_it']:",recipe['text_it'])
    fix_overlaps(recipe['entities_it'], recipe['text_it'])
    if are_spans_overlapping(recipe['entities_it'], recipe['text_it']):
        print('error at', i)
        # print(recipe['entities_it'])
        error_count += 1
    # for entity in recipe['entities_it']:
    #     print(recipe['text_it'][entity[0]:entity[1]],
    #           end=' '
    #           )
    # print()
print()
print('error_count:',error_count)

overlapping spans: [149, 150, 'QUANTITY'] [149, 150, 'QUANTITY']
first span text: 1
second span text: 1
overlapping text: 1
error at 4
overlapping spans: [3, 8, 'PART'] [3, 17, 'PART']
first span text: cosce
second span text: cosce di pollo
overlapping text: cosce di pollo
error at 5
overlapping spans: [77, 87, 'UNIT'] [77, 87, 'UNIT']
first span text: cucchiaino
second span text: cucchiaino
overlapping text: cucchiaino
error at 6
overlapping spans: [2, 11, 'UNIT'] [2, 11, 'UNIT']
first span text: barattolo
second span text: barattolo
overlapping text: barattolo
error at 13
overlapping spans: [0, 1, 'QUANTITY'] [0, 1, 'QUANTITY']
first span text: 1
second span text: 1
overlapping text: 1
error at 14
overlapping spans: [32, 43, 'UNIT'] [32, 43, 'UNIT']
first span text: sgocciolato
second span text: sgocciolato
overlapping text: sgocciolato
error at 19
overlapping spans: [126, 127, 'QUANTITY'] [126, 127, 'QUANTITY']
first span text: 2
second span text: 2
overlapping text: 2
error at 20
o

In [ ]:
import json

def are_spans_overlapping(spans):
    spans = [(spans[i][0], spans[i][1]) for i in range(len(spans))]
    # Sort the spans based on the start index
    sorted_spans = sorted(spans, key=lambda x: x[0])

    # Iterate through the sorted spans
    for i in range(1, len(sorted_spans)):
        # Check if the current span overlaps with the previous span
        if sorted_spans[i][0] < sorted_spans[i - 1][1]:
            return True

    return False

path = '/home/pgajo/working/food/data/TASTEset/data/TASTEset_semicolon_formatted_en-it_unaligned_aligned_model=mdeberta-v3-base-xl-wa_recipe_aligner_5epochs_error_rate=0.0119.json'
with open(path) as f:
    data = json.load(f)
# print(len(data['annotations']))
error_count = 0
for i, recipe in enumerate(data['annotations']):
    # print('***', i)
    # print("recipe['text_en']:",recipe['text_en'])
    if are_spans_overlapping(recipe['entities_en']):
        print('overlapping spans at recipe',i)
        # print(recipe['entities_en'])
        error_count += 1
    # for entity in recipe['entities_en']:
    #     print(recipe['text_en'][entity[0]:entity[1]],end=' ')
    # print()
    print("recipe['text_it']:",recipe['text_it'])
    if are_spans_overlapping(recipe['entities_it']):
        print('overlapping spans at recipe',i)
        # print(recipe['entities_it'])
        error_count += 1
    # for entity in recipe['entities_it']:
    #     print(recipe['text_it'][entity[0]:entity[1]],
    #           end=' '
    #           )
    # print()
print('error_count:',error_count)

recipe['text_it']: 5 once di rum;4 once di triple sec;3 once di Tia Maria;20 once di succo d'arancia
recipe['text_it']: 2 tubi di cinnamon roll, refrigerati, con glassa;4 cucchiai di burro, fuso;6 uova;1/2 tazza di latte;2 cucchiaini di cannella;2 cucchiaini di vaniglia;1 tazza di sciroppo d'acero
overlapping spans at recipe 1
recipe['text_it']: 4 noci di cocco mature;1 tazza di latte evaporato;1 tazza di gin;3 cucchiai di zucchero (facoltativo);1 cucchiaino di cannella macinata;1/2 cucchiaino di noce moscata fresca grattugiata
recipe['text_it']: 1 foglio di graham cracker (spezzato a metà);2 barrette di cioccolato al latte;1 marshmallow;6-8 mirtilli freschi;2 fette di fragola spesse
overlapping spans at recipe 3
recipe['text_it']: 1 confezione da 8 once di involtini primavera;8 fette di prosciutto crudo;4 cucchiaini di senape preparata;1 tazza di formaggio svizzero a scaglie o 1 tazza di formaggio cheddar;2 cucchiai di semi di sesamo (facoltativo)
overlapping spans at recipe 4
recipe[

In [18]:
import json

def adjust_overlapping_spans(spans, text):
    # Sort the spans based on the start index
    sorted_spans = sorted(spans, key=lambda x: x[0])
    adjusted_spans = [sorted_spans[0]]  # Start with the first span

    for i in range(1, len(sorted_spans)):
        prev_span = adjusted_spans[-1]
        curr_span = sorted_spans[i]

        if curr_span[0] < prev_span[1]:
            # Overlapping, adjust the end of the previous span
            new_end = curr_span[0] - 1
            adjusted_spans[-1] = (prev_span[0], new_end, prev_span[2])

        adjusted_spans.append(curr_span)

    return adjusted_spans

# Path to your JSON file
path = '/home/pgajo/working/food/data/TASTEset/data/TASTEset_semicolon_formatted_en-it_unaligned_aligned_model=mdeberta-v3-base-xl-wa_recipe_aligner_5epochs_error_rate=0.0119.json'
new_path = path.replace('.json', '_corrected.json')

with open(path) as f:
    data = json.load(f)

for i, recipe in enumerate(data['annotations']):
    # Adjust English and Italian spans
    recipe['entities_en'] = adjust_overlapping_spans(recipe['entities_en'], recipe['text_en'])
    recipe['entities_it'] = adjust_overlapping_spans(recipe['entities_it'], recipe['text_it'])

# Save the adjusted data to a new JSON file
with open(new_path, 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Data saved to {new_path}")

Data saved to /home/pgajo/working/food/data/TASTEset/data/TASTEset_semicolon_formatted_en-it_unaligned_aligned_model=mdeberta-v3-base-xl-wa_recipe_aligner_5epochs_error_rate=0.0119_corrected.json
